In [62]:
# importing required modules
import PyPDF2
import re
import pandas as pd
	
# creating a pdf file object
pdfFileObj = open('Abaji anandrao jadhav cibill report.pdf', 'rb')
	
# creating a pdf reader object
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
	
# # printing number of pages in pdf file
# print(pdfReader.numPages)
	
# # creating a page object
# pageObj = pdfReader.getPage(0)
	
# # extracting text from page
# print(pageObj.extractText())
	
# # closing the pdf file object
# pdfFileObj.close()



In [63]:
balance = 3009
openAcc = True

In [64]:
def get_index(firstIndex,string):
    return int(firstIndex)+int(len(string))

In [69]:
page2Obj = pdfReader.getPage(1)
print(page2Obj.extract_text())

Page 2 Copyright © 2019 Equifax Credit Information Services Private LimitedReport Order No: 606121585 Date  	 : 06-12-2022
Time 	 : 13:25:48Equifax Credit Report with Score
Account Details :
Accounts
Acct # : 077273410000022 Balance: Rs. 8,12,065 Open: Yes Date Reported: 31-08-2022
Institution : Bank of India Past Due Amount: 0 Interest Rate: 10.75 Date Opened: 12-06-2019
Type: Mudra Loans - Shishu / Kishor / 
TarunLast Payment: Last Payment Date: 17-06-2021 Date Closed:
Ownership Type: Individual Write-off Amount: Sanction Amount : Rs. 9,00,000 Reason:
Repayment Tenure: 45 Monthly Payment Amount: Rs. 19,784 Credit Limit: Collateral Value:
Dispute Code: Term Frequency: Monthly Collateral Type: Property
Account Status: Restructured Due to COVID19
Asset Classification: Standard
Suit Filed Status:
History
Account Status:
Asset Classification:
Suit Filed Status:RCV
STD
*
08-22*
*
*
07-22RCV
STD
*
06-22RCV
STD
*
05-22RCV
STD
*
04-22RCV
STD
*
03-22RCV
STD
*
02-22RCV
STD
*
01-22RCV
STD
*
12-2

In [121]:
def create_loan(text,on_page_2,completeDF):
    completeDF = {"type":[],"institution":[],"sanction_credit":[],"balance":[],"emi":[],"paid_principle":[],"open":[]}
    countAcc = text.count("Acct # :")
    for _ in range(countAcc):
        loan = dict()
        accountIndex = text.find("Acct # :")
        text = text[accountIndex+1:]
        openIndex = get_index(text.find('Open: '),'Open: ')
        openValue = text[openIndex:(text.find("Date Reported: "))].strip()
        # print(text)
        if(on_page_2):
            balanceIndex = get_index(text.find('Balance: '),'Balance: ')
            balance = text[balanceIndex:(text.find('Open:'))]
            if(balance[0] == "R"):
                balance = balance[4:]
            balance = int(balance.replace(',',''))
        # Find institution
        instiutionIndex = get_index(text.find('Institution : '),'Institution : ')
        instiutionName = text[instiutionIndex:(text.find('Past Due Amount'))-1]

        # Find type of loan
        typeIndex = get_index(text.find('Type: '),'Type: ')
        typeName = text[typeIndex:(text.find('Last Payment:'))-1]

        # Find EMI
        emiIndex = get_index(text.find('Monthly Payment Amount: '),'Monthly Payment Amount:')
        emiValue = text[emiIndex:(text.find('Credit Limit:'))-1]
        if(emiValue == ''):
            emiValue = 0
        else:
            emiValue = emiValue[4:]
            try:
                emiValue = int(emiValue.replace(',',''))
            except ValueError:
                emiValue = 0
        
        if(typeName == 'Credit Card'):
            creditIndex = get_index(text.find('Credit Limit:'),"Credit Limit: Rs. ")
            creditLimit = text[creditIndex:text.find('Collateral Value')-1]
            sanction_credit = int(creditLimit.replace(',',''))
        else:
            sanctionIndex = get_index(text.find('Sanction Amount :'),"Sanction Amount : ")
            sanctionLimit = text[sanctionIndex:text.find('Reason:')]
            # print(sanctionLimit)
            if(sanctionLimit == ''):
                sanction_credit = 0
            else:
                sanctionLimit = sanctionLimit[4:]
                try:
                    sanction_credit = int(sanctionLimit.replace(',',''))
                except ValueError:
                    sanction_credit = 0
        completeDF['balance'].append(int(balance))
        completeDF['institution'].append(instiutionName)
        completeDF['type'].append(typeName)
        completeDF['sanction_credit'].append(int(sanction_credit))
        completeDF['emi'].append(int(emiValue))
        completeDF['paid_principle'].append(int(sanction_credit-balance))
        completeDF['open'].append(openValue)
    return completeDF

In [122]:
finalDF = {"type":[],"institution":[],"sanction_credit":[],"balance":[],"emi":[],"paid_principle":[],"open":[]}
for i in range(pdfReader.numPages):
    pageObj = pdfReader.getPage(i)
    newDF = create_loan(pageObj.extract_text(),True,finalDF)
    [finalDF["type"].append(i) for i in newDF["type"]]
    [finalDF["institution"].append(i) for i in newDF["institution"]]
    [finalDF["sanction_credit"].append(i) for i in newDF["sanction_credit"]]
    [finalDF["balance"].append(i) for i in newDF["balance"]]
    [finalDF["emi"].append(i) for i in newDF["emi"]]
    [finalDF["paid_principle"].append(i) for i in newDF["paid_principle"]]
    [finalDF['open'].append(i) for i in newDF["open"]]

In [123]:
data_df = pd.DataFrame.from_dict(finalDF)
data_df

,type,institution,sanction_credit,balance,emi,paid_principle,open
0,Mudra Loans - Shishu / Kishor / \nTaru,Bank of India,900000,812065,19784,87935,Yes
1,Consumer Loan,DMI Finance Private Limited,8896,5560,0,3336,Yes
2,Overdraft,ICICI BANK LIMITED,0,-572,0,572,Yes
3,Personal Loan,Si Creva capital Services \nPrivate Limite,1426,1426,1426,0,Yes
4,Gold Loan,Bank of India,127300,134597,0,-7297,Yes
5,Gold Loan,Bank of India,99000,109311,0,-10311,Yes
6,Gold Loan,Bank of India,33700,27109,0,6591,Yes
7,Mudra Loans - Shishu / Kishor / \nTaru,Bank of India,145000,0,2417,145000,Yes
8,Mudra Loans - Shishu / Kishor / \nTaru,Bank of India,86500,47871,1442,38629,Yes
9,Auto Loan,ICICI BANK LIMITED,915000,0,0,915000,No


In [124]:
data_df['open'] = data_df['open'].map({
    'Yes': True,
    'No': False
})
data_df

,type,institution,sanction_credit,balance,emi,paid_principle,open
0,Mudra Loans - Shishu / Kishor / \nTaru,Bank of India,900000,812065,19784,87935,True
1,Consumer Loan,DMI Finance Private Limited,8896,5560,0,3336,True
2,Overdraft,ICICI BANK LIMITED,0,-572,0,572,True
3,Personal Loan,Si Creva capital Services \nPrivate Limite,1426,1426,1426,0,True
4,Gold Loan,Bank of India,127300,134597,0,-7297,True
5,Gold Loan,Bank of India,99000,109311,0,-10311,True
6,Gold Loan,Bank of India,33700,27109,0,6591,True
7,Mudra Loans - Shishu / Kishor / \nTaru,Bank of India,145000,0,2417,145000,True
8,Mudra Loans - Shishu / Kishor / \nTaru,Bank of India,86500,47871,1442,38629,True
9,Auto Loan,ICICI BANK LIMITED,915000,0,0,915000,False


In [126]:
resultsDF = data_df.loc[data_df['open']==True]
resultsDF

,type,institution,sanction_credit,balance,emi,paid_principle,open
0,Mudra Loans - Shishu / Kishor / \nTaru,Bank of India,900000,812065,19784,87935,True
1,Consumer Loan,DMI Finance Private Limited,8896,5560,0,3336,True
2,Overdraft,ICICI BANK LIMITED,0,-572,0,572,True
3,Personal Loan,Si Creva capital Services \nPrivate Limite,1426,1426,1426,0,True
4,Gold Loan,Bank of India,127300,134597,0,-7297,True
5,Gold Loan,Bank of India,99000,109311,0,-10311,True
6,Gold Loan,Bank of India,33700,27109,0,6591,True
7,Mudra Loans - Shishu / Kishor / \nTaru,Bank of India,145000,0,2417,145000,True
8,Mudra Loans - Shishu / Kishor / \nTaru,Bank of India,86500,47871,1442,38629,True
15,Business Loan,Bajaj Finance Limited,516000,378738,0,137262,True
